# 🌦️ Weather Effects Simulation

Demonstrate physics-based weather simulation for testing sensor robustness.

## Weather Conditions

- 🌫️ **Fog**: Gaussian blur + brightness reduction + LiDAR attenuation
- 🌧️ **Rain**: Noise addition + contrast change + streak effects
- 🌙 **Low Light**: Brightness reduction + gamma correction + noise
- ☀️ **Clear**: Baseline condition for comparison

In [ ]:
# Setup
!git clone https://github.com/cclasher916-spec/camera-lidar-fusion.git
import os
os.chdir('/content/camera-lidar-fusion')
!pip install -q -r requirements.txt

import numpy as np
import matplotlib.pyplot as plt
import cv2

from src.data.data_loader import KITTIDataLoader
from src.simulation.weather_effects import WeatherSimulator

print('✅ Setup complete!')

In [ ]:
# Load sample data
data_loader = KITTIDataLoader({'dataset_path': 'data/kitti', 'max_samples': 5})
sample = data_loader.load_sample(0)

# Initialize weather simulator
weather_config = {
    'effects': {
        'fog': {
            'blur_kernel': 5,
            'brightness_factor': 0.7,
            'contrast_factor': 0.8,
            'lidar_attenuation': 0.9
        },
        'rain': {
            'noise_std': 15,
            'brightness_factor': 0.8,
            'contrast_factor': 1.2,
            'streak_density': 100,
            'lidar_noise_factor': 1.1
        },
        'low_light': {
            'brightness_factor': 0.3,
            'gamma': 1.5,
            'noise_std': 10,
            'lidar_intensity_factor': 0.8
        }
    }
}

weather_sim = WeatherSimulator(weather_config)
print('🌦️ Weather simulator ready!')

In [ ]:
# Demonstrate all weather effects
conditions = ['clear', 'fog', 'rain', 'low_light']

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

for i, condition in enumerate(conditions):
    # Apply weather effect
    modified_data = weather_sim.apply_weather(sample, condition)
    
    # Camera effects
    axes[0, i].imshow(modified_data['camera_image'])
    axes[0, i].set_title(f'📷 {condition.replace("_", " ").title()}')
    axes[0, i].axis('off')
    
    # LiDAR point cloud visualization
    points = modified_data['lidar_points']
    axes[1, i].scatter(points[:, 0], points[:, 1], 
                      c=points[:, 3], s=0.1, cmap='viridis')
    axes[1, i].set_title(f'🎯 LiDAR - {condition.replace("_", " ").title()}')
    axes[1, i].set_xlabel('X (m)')
    axes[1, i].set_ylabel('Y (m)')
    axes[1, i].axis('equal')

plt.suptitle('🌦️ Weather Effects on Camera and LiDAR Data', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Detailed fog effect analysis
print('🌫️ Analyzing Fog Effects...')

fog_intensities = [0.3, 0.5, 0.7, 0.9]

plt.figure(figsize=(20, 5))

for i, intensity in enumerate(fog_intensities):
    # Create custom fog config
    custom_config = {
        'effects': {
            'fog': {
                'blur_kernel': int(3 + intensity * 10),
                'brightness_factor': intensity,
                'contrast_factor': intensity,
                'lidar_attenuation': intensity
            }
        }
    }
    
    temp_sim = WeatherSimulator(custom_config)
    foggy_data = temp_sim.apply_weather(sample, 'fog')
    
    plt.subplot(1, 4, i+1)
    plt.imshow(foggy_data['camera_image'])
    plt.title(f'Fog Intensity: {intensity}')
    plt.axis('off')

plt.suptitle('🌫️ Fog Intensity Progression', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Rain effect analysis
print('🌧️ Analyzing Rain Effects...')

rain_intensities = ['light', 'moderate', 'heavy', 'extreme']
rain_configs = [
    {'noise_std': 5, 'streak_density': 25},
    {'noise_std': 10, 'streak_density': 50},
    {'noise_std': 20, 'streak_density': 100},
    {'noise_std': 30, 'streak_density': 200}
]

plt.figure(figsize=(20, 5))

for i, (intensity, config) in enumerate(zip(rain_intensities, rain_configs)):
    # Create custom rain config
    custom_config = {
        'effects': {
            'rain': {
                'noise_std': config['noise_std'],
                'brightness_factor': 0.8,
                'contrast_factor': 1.2,
                'streak_density': config['streak_density'],
                'lidar_noise_factor': 1.0 + config['noise_std'] / 100
            }
        }
    }
    
    temp_sim = WeatherSimulator(custom_config)
    rainy_data = temp_sim.apply_weather(sample, 'rain')
    
    plt.subplot(1, 4, i+1)
    plt.imshow(rainy_data['camera_image'])
    plt.title(f'Rain: {intensity.title()}')
    plt.axis('off')

plt.suptitle('🌧️ Rain Intensity Progression', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Low light conditions
print('🌙 Analyzing Low Light Effects...')

light_levels = ['dusk', 'twilight', 'night', 'dark_night']
brightness_factors = [0.6, 0.4, 0.2, 0.1]

plt.figure(figsize=(20, 5))

for i, (level, brightness) in enumerate(zip(light_levels, brightness_factors)):
    # Create custom low light config
    custom_config = {
        'effects': {
            'low_light': {
                'brightness_factor': brightness,
                'gamma': 1.5,
                'noise_std': 10 + (1 - brightness) * 20,
                'lidar_intensity_factor': 0.8
            }
        }
    }
    
    temp_sim = WeatherSimulator(custom_config)
    dark_data = temp_sim.apply_weather(sample, 'low_light')
    
    plt.subplot(1, 4, i+1)
    plt.imshow(dark_data['camera_image'])
    plt.title(f'{level.replace("_", " ").title()}\nBrightness: {brightness}')
    plt.axis('off')

plt.suptitle('🌙 Light Level Progression', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Quantitative analysis of weather effects
print('📊 Quantitative Weather Effect Analysis...')

conditions = ['clear', 'fog', 'rain', 'low_light']
metrics = {
    'mean_brightness': [],
    'std_brightness': [],
    'contrast': [],
    'lidar_point_count': [],
    'lidar_mean_intensity': []
}

for condition in conditions:
    modified_data = weather_sim.apply_weather(sample, condition)
    
    # Camera metrics
    img = modified_data['camera_image']
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    metrics['mean_brightness'].append(np.mean(gray))
    metrics['std_brightness'].append(np.std(gray))
    metrics['contrast'].append(np.std(gray) / np.mean(gray) if np.mean(gray) > 0 else 0)
    
    # LiDAR metrics
    points = modified_data['lidar_points']
    metrics['lidar_point_count'].append(len(points))
    metrics['lidar_mean_intensity'].append(np.mean(points[:, 3]))

# Create metrics visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Camera metrics
axes[0, 0].bar(conditions, metrics['mean_brightness'], color='blue', alpha=0.7)
axes[0, 0].set_title('Mean Brightness')
axes[0, 0].set_ylabel('Brightness (0-255)')
axes[0, 0].tick_params(axis='x', rotation=45)

axes[0, 1].bar(conditions, metrics['std_brightness'], color='green', alpha=0.7)
axes[0, 1].set_title('Brightness Std Dev')
axes[0, 1].set_ylabel('Standard Deviation')
axes[0, 1].tick_params(axis='x', rotation=45)

axes[0, 2].bar(conditions, metrics['contrast'], color='red', alpha=0.7)
axes[0, 2].set_title('Contrast Ratio')
axes[0, 2].set_ylabel('Std/Mean')
axes[0, 2].tick_params(axis='x', rotation=45)

# LiDAR metrics
axes[1, 0].bar(conditions, metrics['lidar_point_count'], color='purple', alpha=0.7)
axes[1, 0].set_title('LiDAR Point Count')
axes[1, 0].set_ylabel('Number of Points')
axes[1, 0].tick_params(axis='x', rotation=45)

axes[1, 1].bar(conditions, metrics['lidar_mean_intensity'], color='orange', alpha=0.7)
axes[1, 1].set_title('Mean LiDAR Intensity')
axes[1, 1].set_ylabel('Intensity Value')
axes[1, 1].tick_params(axis='x', rotation=45)

# Combined degradation
brightness_degradation = [(metrics['mean_brightness'][0] - b) / metrics['mean_brightness'][0] * 100 
                         for b in metrics['mean_brightness']]
lidar_degradation = [(metrics['lidar_point_count'][0] - p) / metrics['lidar_point_count'][0] * 100 
                    for p in metrics['lidar_point_count']]

x = np.arange(len(conditions))
width = 0.35

axes[1, 2].bar(x - width/2, brightness_degradation, width, label='Camera', alpha=0.7)
axes[1, 2].bar(x + width/2, lidar_degradation, width, label='LiDAR', alpha=0.7)
axes[1, 2].set_title('Performance Degradation')
axes[1, 2].set_ylabel('Degradation (%)')
axes[1, 2].set_xticks(x)
axes[1, 2].set_xticklabels(conditions, rotation=45)
axes[1, 2].legend()
axes[1, 2].axhline(y=0, color='black', linestyle='-', alpha=0.5)

plt.suptitle('📊 Quantitative Weather Impact Analysis', fontsize=16)
plt.tight_layout()
plt.show()

# Print summary
print('
📋 Weather Impact Summary:')
for condition in conditions[1:]:  # Skip clear
    idx = conditions.index(condition)
    brightness_loss = (metrics['mean_brightness'][0] - metrics['mean_brightness'][idx]) / metrics['mean_brightness'][0] * 100
    point_loss = (metrics['lidar_point_count'][0] - metrics['lidar_point_count'][idx]) / metrics['lidar_point_count'][0] * 100
    
    print(f'  {condition.title()}:')
    print(f'    Camera brightness loss: {brightness_loss:.1f}%')
    print(f'    LiDAR point loss: {point_loss:.1f}%')

## 🔍 Weather Effect Analysis

### Camera Effects
- **Fog**: Reduces visibility through blur and brightness reduction
- **Rain**: Adds noise and streaking, affects contrast
- **Low Light**: Drastically reduces brightness, increases noise

### LiDAR Effects
- **Fog**: Attenuates laser returns, reduces detection range
- **Rain**: Adds noise to point cloud, false returns from droplets
- **Low Light**: Minimal direct effect (active sensor)

### Fusion Motivation
Different sensors are affected differently by weather conditions:
- Camera severely degraded by fog and low light
- LiDAR more robust but affected by precipitation
- Fusion can leverage the strengths of each sensor under different conditions